In [1]:
## reference https://huggingface.co/learn/nlp-course/en/chapter7/6?fw=pt#training-a-causal-language-model-from-scratch

In [2]:
#installing some libraries
#!pip install datasets
!pip install --upgrade jupyter ipywidgets



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch, transformers
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import PreTrainedModel, PretrainedConfig
from transformers import AutoModel, AutoConfig,AutoModelForCausalLM, GPT2Config,GPT2Tokenizer,GPT2LMHeadModel
from transformers import BertModel, BertTokenizer, GPT2LMHeadModel,BertConfig,DistilBertModel, DistilBertTokenizer,DistilBertConfig
import torch.nn as nn
import torch.nn.functional as F
#from datasets import load_dataset
import pandas as pd, numpy as np
from torch.cuda.amp import autocast
from torch import cuda
import datetime
import warnings,itertools
from torch.optim.lr_scheduler import LambdaLR
import json
import random
# Ignore all warnings
warnings.filterwarnings('ignore')
#pip install transformers bitsandbytes>=0.39.0 -q
import zipfile,logging

In [4]:
from transformers import set_seed

def set_global_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    set_seed(seed)  # This sets the seed for transformers library

# Set the seed
seed = 42
set_global_seed(seed)


In [5]:
#global params for training

B =  4
T_g = 1024
T_b = 512
epoch = 100
random_init_wts = True
# hard coded compression ratio. This is the ratio of text_len to the # of tokens produced by the tokeinzer
comp_ratio = 3

if cuda.is_available():
    device = torch.device('cuda:0')
    print(device)
else:
    device = 'cpu'
#print(device)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
#os.environ["MKL_DEBUG_CPU_TYPE"] = "5"

#print(global_tr_loss)
model_path = os.path.join("model")
print(model_path)

cuda:0
model


In [6]:
directory = os.path.join('.','data','unzip_text_10M')  # Replace with your directory path
directory

'./data/unzip_text_10M'

In [7]:
def read_text(directory):
    directory = os.path.join('.','data','unzip_text_10M',str(directory))  # Replace with your directory path
    print(f"directory :{directory}")
    # List all files in the directory
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    print(f"files:{files}")
    text_content = []
    # Read each file
    total_lines = 0
    for filenum,filename in enumerate(files):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            #first_line = file.read()
            #print(f"filename :{filename}->first few lines {first_line}")
            #continue
            #lines_list = [line.strip() for line in open(file_path, 'r')]
            text = file.read()
            text_content.append(text)
            print(f"the file:{filename} has been appeneded to the uber list and its length is {len(text_content)} ")
            #total_lines+=len(lines_list)
            #text_content.append(lines_list)
    
    flattened_list = ''.join(text_content)
    assert (len(flattened_list) == total_lines , f"Expected {len(flattened_list)} to be equal to {total_lines}" )
    
    return flattened_list

    

In [8]:
train_list = read_text("train_10M")


directory :./data/unzip_text_10M/train_10M
files:['switchboard.train', 'simple_wiki.train', 'open_subtitles.train', 'gutenberg.train', 'childes.train', 'bnc_spoken.train']
the file:switchboard.train has been appeneded to the uber list and its length is 1 
the file:simple_wiki.train has been appeneded to the uber list and its length is 2 
the file:open_subtitles.train has been appeneded to the uber list and its length is 3 
the file:gutenberg.train has been appeneded to the uber list and its length is 4 
the file:childes.train has been appeneded to the uber list and its length is 5 
the file:bnc_spoken.train has been appeneded to the uber list and its length is 6 


In [9]:
len(train_list)

54215049

In [10]:
val_list = read_text("dev")


directory :./data/unzip_text_10M/dev
files:['switchboard.dev', 'simple_wiki.dev', 'open_subtitles.dev', 'gutenberg.dev', 'childes.dev', 'bnc_spoken.dev']
the file:switchboard.dev has been appeneded to the uber list and its length is 1 
the file:simple_wiki.dev has been appeneded to the uber list and its length is 2 
the file:open_subtitles.dev has been appeneded to the uber list and its length is 3 
the file:gutenberg.dev has been appeneded to the uber list and its length is 4 
the file:childes.dev has been appeneded to the uber list and its length is 5 
the file:bnc_spoken.dev has been appeneded to the uber list and its length is 6 


In [11]:
# # Test the tokenizer:
bert_model_name = 'distilbert-base-uncased'
gpt2_model_name = 'distilgpt2'
bert_tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name)
bert_config =  DistilBertConfig()
bert_model = DistilBertModel(bert_config)
gpt2_config = AutoConfig.from_pretrained(gpt2_model_name, vocab_size = 50304)
gpt_model = AutoModelForCausalLM.from_config(gpt2_config)


In [12]:
gpt_tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
if bert_tokenizer.pad_token is None:
    bert_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [13]:
#DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [14]:
bert_tokenizer.pad_token

'[PAD]'

In [15]:
# class BertGPT2EncoderDecoder_bkp(torch.nn.Module):
#     def __init__(self, bert_model, gpt_model):
#         super(BertGPT2EncoderDecoder, self).__init__()
#         self.bert_model = bert_model
#         self.gpt_model = gpt_model
        
#         # Linear layer to match BERT output dim to GPT-2 input dim
#         self.linear = torch.nn.Linear(bert_model.config.hidden_size, gpt_model.config.n_embd)

#     def forward(self, bet_enc,gpt_enc, labels=None):
#         # Encode with BERT
#         bert_input_id = bet_enc['input_ids']
#         bert_attention = bet_enc['attention_mask']
        
#         gpt_input_id = gpt_enc['input_ids']
#         gpt_attention = gpt_enc['attention_mask']
        
#         bert_input_id = bert_input_id.view(B,T_b)
#         bert_attention = bert_attention.view(B,T_b)
        
#         print(f"shape that goes into the model | input_id = {bert_input_id.shape}|attention_mask = {bert_attention.shape}")
        
#         encoder_outputs = self.bert_model(input_ids=bert_input_id, attention_mask=bert_attention)
#         encoder_hidden_states = encoder_outputs.last_hidden_state  # Shape: [batch_size, seq_len, bert_hidden_size]
#         print(f"shape of Bert output = {encoder_hidden_states.shape}")
                
#         # Project BERT output to GPT-2 input dimension
#         projected_encoder_hidden_states = self.linear(encoder_hidden_states)  # Shape: [batch_size, seq_len, gpt2_hidden_size]
#         print(f"After linear layer shape = {projected_encoder_hidden_states.shape}")
#         # Decode with GPT-2
#         extended_hidden_states = torch.cat([projected_encoder_hidden_states] * 2, dim=1)  # Shape: [batch_size, 1024, gpt2_hidden_size]
#         # Extend attention mask
#         extended_attention_mask = torch.cat([bert_attention] * 2, dim=1)  # Shape: [batch_size, 1024]
#         print(f"shape of extended_hidden_sate = {extended_hidden_states.shape}| extended attenion shape = {extended_attention_mask.shape}")
#         labels = torch.cat([bert_input_id] * 2, dim=1).view(B,T_g)
        
        
#         decoder_outputs = self.gpt_model(inputs_embeds=extended_hidden_states, attention_mask = extended_attention_mask,
#                                           labels=labels)
#         #print(f"decoder_outputs = {decoder_outputs}")

        

#         return decoder_outputs

In [16]:
def tokenize_text (chunk, bert_tokenizer = bert_tokenizer,bert_max_length = B*T_b,gpt_tokenizer = gpt_tokenizer,gpt_max_length = B*T_g):
    
    
    enc_bert = bert_tokenizer(chunk,padding='max_length',truncation=True,max_length=gpt_max_length,return_tensors="pt",return_attention_mask=True)
    #enc_gpt = gpt_tokenizer(chunk,padding='max_length',truncation=True,max_length=gpt_max_length,return_tensors="pt",return_attention_mask=True)
    
    label_enc =gpt_tokenizer(chunk,padding='max_length',truncation=True,max_length=gpt_max_length,return_tensors="pt",return_attention_mask=True)
    labels = label_enc['input_ids']
    #labels[labels == gpt_tokenizer.pad_token_id] = -100  # Ignore padding tokens in the loss
    #print(f"attention mask = {enc_bert['attention_mask'].shape}| bert_input_id_shape = {label_enc['input_ids'].shape}")   
    
    return enc_bert,labels
    
    
    
        
    


### Data loaders and Dataset for batched training

In [17]:
class dataset_pyt_train(Dataset):
    def __init__(self, text_list, B = B, bert_tokenizer = bert_tokenizer, T_b = T_b,comp_ratio = comp_ratio):
        self.text_list = text_list
        self.bert_tokeinzer = bert_tokenizer     
        
                                        
    def __getitem__(self, idx):
        
        chunk = self.text_list[idx:idx + B*T_g*comp_ratio]
        enc_bert,labels = tokenize_text(chunk)
              
        return enc_bert,labels
        
        
    def __len__(self):
        
        num_chunks = comp_ratio*B*T_g
        return len(self.text_list)//num_chunks

In [18]:
class dataset_pyt_val(Dataset):
    def __init__(self, text_list, B = B, bert_tokenizer = bert_tokenizer, T_b = T_b,comp_ratio = comp_ratio):
        self.text_list = text_list
        self.bert_tokeinzer = bert_tokenizer     
        
                                        
    def __getitem__(self, idx):
        
        chunk = self.text_list[idx:idx + B*T_g*comp_ratio]
        enc_bert,labels = tokenize_text(chunk)
                
        return enc_bert,labels
        
        
    def __len__(self):
        
        num_chunks = comp_ratio*B*T_g
        return len(self.text_list)//num_chunks

In [19]:


class BertGPT2EncoderDecoder(PreTrainedModel):
    def __init__(self, bert_model, gpt_model):
        config = bert_model.config
        super().__init__(config)
        self.bert_model = bert_model
        self.gpt_model = gpt_model
        
        # Linear layer to match BERT output dim to GPT-2 input dim
        self.linear = torch.nn.Linear(bert_model.config.hidden_size, gpt_model.config.n_embd)

    def forward(self, bert_ids,bert_att_mask,labels):
        #bert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')     
        bert_input_id = bert_ids.view(B, T_g)
        bert_attention = bert_att_mask.view(B, T_g)
        assert not torch.isnan(bert_input_id).any(), "Input IDs contain NaNs"
        assert not torch.isinf(bert_input_id).any(), "Input IDs contain Infs"
        #we split the 1024 dim tensor into two, each of 512 len 
        split_tensors_inp = torch.split(bert_input_id, T_b, dim=1)
        bert_input_id_chunk1,bert_input_id_chunk2 = split_tensors_inp
                
        split_tensors_att = torch.split(bert_attention, T_b, dim=1)
        bert_att_chunk1,bert_att_chunk2 = split_tensors_att
               
                 
        encoder_outputs_chunk_1 = self.bert_model(input_ids=bert_input_id_chunk1, attention_mask=bert_att_chunk1)
        encoder_outputs_chunk_2 = self.bert_model(input_ids=bert_input_id_chunk2, attention_mask=bert_att_chunk2)
#         print(f"encoder_outputs_chunk_1 = {encoder_outputs_chunk_1}")
#         print(f"encoder_outputs_chunk_2 = {encoder_outputs_chunk_2}")
        
        encoder_hidden_states_1 = encoder_outputs_chunk_1.last_hidden_state
        encoder_hidden_states_2 = encoder_outputs_chunk_2.last_hidden_state# Shape: [batch_size, seq_len, bert_hidden_size]
        
#         encoder_hidden_states_1[bert_input_id_chunk1 == bert_tokenizer.pad_token_id] = 0
#         encoder_hidden_states_2[bert_input_id_chunk2 == bert_tokenizer.pad_token_id] = 0
        
        extended_hidden_states = torch.cat((encoder_hidden_states_1, encoder_hidden_states_2),dim=1)  # Shape: [batch_size, 1024, gpt2_hidden_size]
        extended_attention_mask = torch.cat((bert_att_chunk1,bert_att_chunk2), dim=1)  # Shape: [batch_size, 1024]
        labels = labels.view(B,T_g)
        
        decoder_outputs = self.gpt_model(inputs_embeds=extended_hidden_states, attention_mask=extended_attention_mask, labels=labels)
        #print(f"decoder out = {decoder_outputs}")
        
        return decoder_outputs

In [20]:
# low = 1
# high = 1000  # high is exclusive in torch.randint, so we use 52548 to include 52547
# shape1 = (32, 1024)
# dtype = torch.long
# shape2 = (32, 1024)
# # Create the random integer tensor

# m1 =  BertGPT2EncoderDecoder(bert_model = bert_model, gpt_model = gpt_model)
# m1.to(device)
# bert_id =  torch.randint(low, high, shape1, dtype=dtype,device = device)
# att = torch.ones(32,1024,device = device)
# labels =  torch.randint(low, high, shape2, dtype=dtype,device = device)
# o = m1(bert_id,att,labels)

In [21]:
model = BertGPT2EncoderDecoder(bert_model = bert_model, gpt_model = gpt_model)
model.to(device)
model = torch.compile(model)

In [22]:
#train_dataset = dataset_pyt(filtered_df,tokenizer = tokenizer)
train_dataset = dataset_pyt_train(train_list)
val_dataset = dataset_pyt_val(val_list)

train_loader = DataLoader(train_dataset,batch_size = 1, shuffle = True , num_workers = 4, pin_memory = True)
val_loader = DataLoader(val_dataset,batch_size = 1, shuffle = True , num_workers = 4, pin_memory = True)


In [23]:
# b1 = torch.rand(64,512,768)
# b1.shape
# g1 = torch.rand(64,512)
# g1.shape
# cat = torch.cat((b1,g1),dim = 1)

In [24]:
enc_bert,labels = train_dataset[1]
bert_ids = enc_bert['input_ids']
bert_att_mask = enc_bert['attention_mask']
bert_ids = bert_ids.to(device=device, non_blocking=True)
bert_ids = torch.squeeze(bert_ids, dim = 0)
bert_att_mask = bert_att_mask.to(device=device, non_blocking=True)
bert_att_mask =  torch.squeeze(bert_att_mask, dim = 0)
labels = labels.to(device=device, non_blocking=True)
labels = torch.squeeze(labels, dim = 0).view(B,T_g)
print(f"bert_ids shape = {bert_ids.shape}|att_mask shape = {bert_att_mask.shape}| labels shape = {labels.shape}")
model.to(device)
#bert_ids,bert_att_mask,ids_gpt,att_gpt, labels

o = model(bert_ids , bert_att_mask,labels)

bert_ids shape = torch.Size([4096])|att_mask shape = torch.Size([4096])| labels shape = torch.Size([4, 1024])


In [26]:
o

CausalLMOutputWithCrossAttentions(loss=tensor(11.0177, device='cuda:0', grad_fn=<CompiledFunctionBackward>), logits=tensor([[[-0.9231, -0.2377, -0.7966,  ...,  0.2173, -0.5901, -0.2208],
         [ 1.2186,  0.1997,  0.8678,  ...,  1.0923, -0.1040,  0.5834],
         [ 0.5836,  0.0927,  0.8907,  ...,  0.9590, -0.2837,  0.8244],
         ...,
         [ 0.7864,  0.2417, -0.0524,  ..., -0.2040,  0.5840, -0.0755],
         [-0.0113, -0.0454,  0.6392,  ...,  0.3787,  0.3700,  0.1832],
         [-0.2516, -1.0520,  0.1311,  ..., -0.6021,  0.0831,  0.0078]],

        [[-0.6449,  0.3711, -0.4210,  ...,  0.7687, -0.0673, -0.3301],
         [ 0.9181, -0.0960,  0.6469,  ..., -0.1686, -0.2845,  0.6112],
         [ 0.3436, -0.0847,  0.5776,  ..., -0.1081, -0.1943, -0.0067],
         ...,
         [ 0.3017,  0.0982,  0.6515,  ...,  0.6147,  0.3636, -0.1744],
         [ 0.1449, -1.1206,  0.7635,  ..., -0.0800,  0.7663, -0.7426],
         [-0.0482, -0.7726, -0.4588,  ..., -0.8856,  0.1110, -0.2821]],



In [27]:
model.to(device)
for i in range(0,len(train_dataset)):
    enc_bert,labels = train_dataset[i]
    bert_ids = enc_bert['input_ids']
    bert_att_mask = enc_bert['attention_mask']
    bert_ids = bert_ids.to(device=device, non_blocking=True)
    bert_ids = torch.squeeze(bert_ids, dim = 0)
    bert_att_mask = bert_att_mask.to(device=device, non_blocking=True)
    bert_att_mask =  torch.squeeze(bert_att_mask, dim = 0)
    #gpt_extraction
    labels = labels.to(device=device, non_blocking=True)
    labels = torch.squeeze(labels, dim = 0).view(B,T_g)
#bert_ids,bert_att_mask,ids_gpt,att_gpt, labels
    with autocast(dtype = torch.bfloat16):
        model_output = model(bert_ids = bert_ids ,bert_att_mask = bert_att_mask,labels = labels)
        total_loss = model_output.loss
        print(f"index = {i}|loss = {total_loss}")
   

index = 0|loss = 11.018465995788574
index = 1|loss = 11.022727012634277
index = 2|loss = 11.010019302368164
index = 3|loss = 11.027156829833984
index = 4|loss = 11.007025718688965
index = 5|loss = 11.006078720092773
index = 6|loss = 11.016663551330566
index = 7|loss = 11.011791229248047
index = 8|loss = 11.016617774963379
index = 9|loss = 11.015914916992188
index = 10|loss = 11.010844230651855
index = 11|loss = 11.016709327697754
index = 12|loss = 11.015594482421875
index = 13|loss = 11.027782440185547
index = 14|loss = 11.011012077331543
index = 15|loss = 11.025705337524414
index = 16|loss = 11.015518188476562
index = 17|loss = 11.009363174438477
index = 18|loss = 11.016922950744629
index = 19|loss = 11.016160011291504
index = 20|loss = 11.024346351623535
index = 21|loss = 11.008858680725098
index = 22|loss = 11.030592918395996
index = 23|loss = 11.009209632873535
index = 24|loss = 11.012463569641113
index = 25|loss = 11.009011268615723
index = 26|loss = 11.012738227844238
index = 27|

KeyboardInterrupt: 

In [ ]:
# test = torch.rand(1,65536)
# test = torch.squeeze(test,dim = 0)
# test = test.view(B,T_g)

In [26]:
def write_file(log_message, model_name = "enc_dec" ,random_init_wts = random_init_wts ):
    current_datetime = datetime.datetime.now()
    # Extract date and time components
    current_date = str(current_datetime.date())
    log_file = model_name +'_nll_two_bert_encoder_'+'random_init_wts'+ '_'+str(random_init_wts)+'_' +current_date+'.log'
    print(f"*****LOGGING INFO IN {log_file}*********")
    filepath = os.path.join("model",log_file)
    logging.basicConfig(filename=filepath, 
                    filemode='a',  # Overwrite the log file each time
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    level=logging.DEBUG)
    logger = logging.getLogger()
    logger.info(log_message)
    
    

    
    

In [27]:
print(f"Length of the train loader is {len(train_loader)}")
print(f"Length of the val loader is {len(val_loader)}")
print(f"num_tokens= {B*T_g*len(train_loader)}")

Length of the train loader is 551
Length of the val loader is 575
num_tokens= 18055168


In [28]:
#emb,att,inp = train_dataset[1]

In [29]:
class check_train_metrics:
    def __init__(self, patience=20, min_delta=0 , B = B, T = T_g,best_loss = torch.inf,early_stop = False):
        self.patience = patience
        self.min_delta = min_delta
        self.train_counter = 0
        self.best_loss = best_loss
        self.early_stop = early_stop
        self.improvement = None

    def __call__(self, loss, epoch , epoch_durn, norm , current_lr, num_token):
        if self.best_loss - loss > self.min_delta:
            
            print(f"training loss has decreased---> reducing the best loss from {self.best_loss:.2f} to {loss:.2f} | throughput = {int(num_token/epoch_durn)} tokens/second | norm = {norm:.4f} | learning rate = {current_lr:.5e}")
            self.best_loss = loss
            self.train_counter = 0
            self.improvement = True
        else:
            self.train_counter += 1
            self.improvement = False
            print(f"No improvement in training  loss-->epoch= {epoch} and best loss is {self.best_loss:.2f}|current_loss = {loss}|counter = {self.train_counter}")
            if self.train_counter >= self.patience:
                self.early_stop = True


In [30]:
class check_val_metrics:
    def __init__(self, patience=10, min_delta=0, best_loss = torch.inf,early_stop = False):
        self.patience = patience
        self.min_delta = min_delta
        self.val_counter = 0
        self.best_loss = best_loss
        self.early_stop = early_stop
        

    def __call__(self, loss, epoch , model, bert_tokenizer):
        if self.best_loss - loss > self.min_delta:
            print(f"Val loss has decreased -->reducing the global validation loss from {self.best_loss:.2f} to {loss:.2f}")
            s1 = (f"Val loss has decreased -->reducing the global validation loss from {self.best_loss:.2f} to {loss:.2f}")
            print(f" validation loss for epoch = {epoch} is {loss:.4f}")
            self.best_loss = loss
            s2 = f" validation loss for epoch = {epoch} is {loss:.4f}"
            print(f" epoch= {epoch} :  val loss is {loss:.4f} ")
            s3 = f" epoch= {epoch} :  val loss is {loss:.4f} "
            #save the model
            # Get the current date and time
            current_datetime = datetime.datetime.now()
            # Extract date and time components
            current_date = str(current_datetime.date())
            current_time = str(current_datetime.time()).split('.')[0]
            file_name = 'model'+ current_date+current_time+'.pth'
            path = os.path.join("model",file_name)
            print(f"saving the model {file_name}")
            s4 = f"saving the model {file_name}"
            #torch.save(model.state_dict(), path)
            model.save_pretrained(path)
            bert_tokenizer.save_pretrained(path)
            log_message = s1+s2+s3+s4
            write_file(log_message)
            self.val_counter = 0
        else:
            self.val_counter += 1
            print(f"No improvement in validation loss-->epoch= {epoch} and best val loss is {self.best_loss:.2f}|current_Val loss = {loss}|val_counter = {self.val_counter}")
            if self.val_counter >= self.patience:
                self.early_stop = True

In [31]:
#model_output = model(input_ids = inp ,attention_mask = att, labels = inp)

In [32]:
@torch.no_grad

def eval_model(val_loader, model, epoch , device = device,tokenizer = bert_tokenizer):
    model.eval()
    model.to(device)
    e = epoch+1
    val_loss_accum = 0.0
    
    print(f"inside validation data for epoch {e}")
    for ind,(enc_bert,labels) in enumerate(val_loader):
        bert_ids = enc_bert['input_ids']
        bert_att_mask = enc_bert['attention_mask']
        bert_ids = bert_ids.to(device=device, non_blocking=True)
        bert_ids = torch.squeeze(bert_ids, dim = 0)
        bert_att_mask = bert_att_mask.to(device=device, non_blocking=True)
        bert_att_mask =  torch.squeeze(bert_att_mask, dim = 0)
        labels = labels.to(device=device, non_blocking=True)
        labels = torch.squeeze(labels, dim = 0).view(B,T_g)
        #bert_ids,bert_att_mask,labels
        with autocast(dtype = torch.bfloat16):
            model_output = model(bert_ids = bert_ids ,bert_att_mask = bert_att_mask,labels = labels)
            total_loss = model_output.loss
            
    
    
        val_loss_accum+= total_loss.detach().item()
        del bert_att_mask,bert_ids,labels,model_output
    return val_loss_accum        
    
    
    


In [33]:

def train_model(train_loader,val_loader,model,num_epoch = epoch,device = device,tokenizer = bert_tokenizer):
    #model.train()
    device = device
    lr_custom = 1e-6
    print(f"inside train model. Device = {device}")
    model.to(device)
    optimizer = torch.optim.AdamW(params =  model.parameters(), lr= lr_custom,fused = True ,weight_decay = .1)
      
    extra_train = .1*num_epoch
    max_train_steps = int(num_epoch +extra_train )
    import time
    from transformers import get_linear_schedule_with_warmup
    total_steps = len(train_loader) * num_epoch
    scheduler_cos = transformers.get_cosine_schedule_with_warmup( optimizer= optimizer, num_warmup_steps =int(total_steps * 0.1) ,num_training_steps= total_steps )
        
    epoch_train_log = []
    epoch_val_log = []
    validate_val_metric = check_val_metrics()
    validate_train_metric = check_train_metrics()
    for i in range (max_train_steps):
        
        epoch_start_time = time.time()
        optimizer.zero_grad(set_to_none=True)
        # we use 2 schedulers - the first LR scheduler uses a cosine decay for 100 epochs the second scheduler takes the last LR from cosine scheduler and then maintains that LR for the next 10 epochs
        if i >= num_epoch:
            optimizer_reduced_lr = torch.optim.AdamW(params =  model.parameters(), lr= current_lr ,fused = True , weight_decay=.1)
            scheduler_constant = transformers.get_constant_schedule_with_warmup( optimizer = optimizer_reduced_lr ,num_warmup_steps = 0, last_epoch = -1 )
        
        epoch_train_loss = 0       
        for ind,(enc_bert,labels) in enumerate(train_loader):
            if ind == int(len(train_loader)/2):
                batch_time = time.time()
                duration = batch_time - epoch_start_time
                print(f"executing epoch:{i+1}, it took {duration/60} mins from beginning of epoch till batch#{ind}")
            
            #bert extraction
            bert_ids = enc_bert['input_ids']
            bert_att_mask = enc_bert['attention_mask']
            bert_ids = bert_ids.to(device=device, non_blocking=True)
            bert_ids = torch.squeeze(bert_ids, dim = 0)
            bert_att_mask = bert_att_mask.to(device=device, non_blocking=True)
            bert_att_mask =  torch.squeeze(bert_att_mask, dim = 0)
            labels = labels.to(device=device, non_blocking=True)
            labels = torch.squeeze(labels, dim = 0).view(B,T_g)
            #bert_ids,bert_att_mask,labels
            with autocast(dtype = torch.bfloat16):
                model_output = model(bert_ids = bert_ids ,bert_att_mask = bert_att_mask,labels = labels)
                total_loss = model_output.loss
            
                    
            total_loss.backward()
            epoch_train_loss += total_loss.detach().item()
            norm = torch.nn.utils.clip_grad_norm(model.parameters() , 1.0)
            if i <= num_epoch:
                optimizer.step()
                scheduler_cos.step()
                optimizer.zero_grad(set_to_none=True)
            else:
                optimizer_reduced_lr.step()
                optimizer_reduced_lr.zero_grad(set_to_none=True)
                scheduler_constant.step()
                
                         
            del bert_att_mask,bert_ids,labels,model_output
            
        #batch processing complete 
        #print(f"batch processing complete , lambda = {lambda_val} |total_loss for batch= {total_loss}")
        
        if i <= num_epoch:
            current_lr = scheduler_cos.get_last_lr()[0]
        epoch_end_time = time.time()
        epoch_durn = (epoch_end_time - epoch_start_time)
        num_token = B*T_g*len(train_loader)
        epoch_train_log.append(epoch_train_loss)
        validate_train_metric(epoch_train_loss, i , epoch_durn, norm , current_lr, num_token)
        
        if validate_train_metric.improvement:
            val_loss= eval_model(val_loader, model, epoch = i, device = device,tokenizer = bert_tokenizer)
            epoch_val_log.append(val_loss)
            validate_val_metric(val_loss, i , model, bert_tokenizer)
            if validate_train_metric.early_stop or validate_val_metric.early_stop :
                print(f"early stopping trigerred either from training data or val data | train_counter = {validate_train_metric.train_counter}|val_counter = {validate_val_metric.val_counter}")
                break
        else:
            if validate_val_metric.early_stop:
                print(f"early stopping trigerred from validation data")
                break
              
    
    return model,epoch_train_log,epoch_val_log
        
            
            
    
    

In [ ]:
tr_model,epoch_train_log,epoch_val_log = train_model(train_loader, val_loader,model=model,tokenizer = bert_tokenizer)

inside train model. Device = cuda:0
executing epoch:1, it took 2.5606335043907165 mins from beginning of epoch till batch#275
training loss has decreased---> reducing the best loss from inf to 5970.55 | throughput = 68387 tokens/second | norm = 3.7244 | learning rate = 1.00000e-07
inside validation data for epoch 1
Val loss has decreased -->reducing the global validation loss from inf to 6048.80
 validation loss for epoch = 0 is 6048.8028
 epoch= 0 :  val loss is 6048.8028 
saving the model model2024-07-3121:50:54.pth
*****LOGGING INFO IN enc_dec_nll_two_bert_encoder_random_init_wts_True_2024-07-31.log*********
executing epoch:2, it took 2.4919617255528768 mins from beginning of epoch till batch#275
training loss has decreased---> reducing the best loss from 5970.55 to 5406.48 | throughput = 70691 tokens/second | norm = 4.7988 | learning rate = 2.00000e-07
inside validation data for epoch 2
Val loss has decreased -->reducing the global validation loss from 6048.80 to 5196.53
 validatio

In [ ]:
64*512

In [ ]:
# import json , os
# path_var_train_log = os.path.join(".","Tokenizers_and_loss_vals","epoch_train_plain_loss.json")
# path_var_val_log = os.path.join(".","Tokenizers_and_loss_vals","epoch_val_val_loss_.json")

# #print(path_var)
# #Write the list to a JSON file
# with open(path_var_train_log, "w") as file:
#     json.dump(epoch_train_log, file)

# with open(path_var_val_log, "w") as file:
#     json.dump(epoch_val_log, file)


In [ ]:
# with open(path_var_train_log, "r") as file:
#     train_loss = json.load(file)
# with open(path_var_val_log, "r") as file:
#     val_loss = json.load(file)

In [ ]:
x_values = range(len(epoch_train_log))
plt.plot(x_values, epoch_train_log, label='Train_loss')

In [ ]:
#epoch_val_log= [t.detach().cpu().numpy() for t in epoch_val_log]
x_values_val = range(len(epoch_val_log))
plt.plot(x_values_val, epoch_val_log, label='val_loss')

In [ ]:
min_length = min(len(epoch_train_log), len(epoch_val_log))
list1 = epoch_train_log[:min_length]
list2 = epoch_val_log[:min_length]

# Create x-axis values
x_values = range(min_length)

# Plot the lists
plt.plot(x_values, list1, label='Train_loss')
plt.plot(x_values, list2, label='Val_loss')

# Adding labels and title
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Train Loss and Val_Loss')
plt.legend()